In [2]:
from day24 import parse, GateType, Gate, Wire, simulate_circuit
import networkx as nx

## Use graphviz to identify the wrong patterns

In [3]:
#type: ignore
from graphviz import Digraph
from IPython.display import display

def visualize_circuit(graph: nx.DiGraph) -> Digraph:
    dot = Digraph(comment='Logic Circuit')
    dot.attr(rankdir='LR')  # Left to right layout

    # Define gate styles
    gate_styles = {
        GateType.AND: {
            'shape': 'record',
            'label': '{{&}}',
            'color': 'red',
            'style': 'filled',
            'fillcolor': '#ffcccc'  # Light red fill
        },
        GateType.OR: {
            'shape': 'record',
            'label': '{{\|}}',
            'color': 'blue',
            'style': 'filled',
            'fillcolor': '#cce6ff'  # Light blue fill
        },
        GateType.XOR: {
            'shape': 'record',
            'label': '{{^}}',
            'color': 'green',
            'style': 'filled',
            'fillcolor': '#ccffcc'  # Light green fill
        }
    }

    # Add nodes
    for node in graph.nodes():
        if isinstance(node, Wire):
            dot.node(
                node.name,
                label=f'{node.name}\n{node.value if node.value is not None else "?"}',
                shape='circle'
            )
        elif isinstance(node, Gate):
            style = gate_styles[node.gate_type]
            dot.node(
                node.name,
                label=style['label'],
                shape=style['shape'],
                color=style['color'],
                style=style['style'],
                fillcolor=style['fillcolor']
            )

    # Add edges
    for edge in graph.edges():
        dot.edge(edge[0].name, edge[1].name)

    return dot


In [ ]:
DIAGRAM = """x00: 1
x01: 1
x02: 1
y00: 0
y01: 1
y02: 0

x00 AND y00 -> z00
x01 XOR y01 -> z01
x02 OR y02 -> z02"""

circuit = parse(DIAGRAM)
dot = visualize_circuit(circuit)
display(dot)

In [ ]:
with open("../data/day24.txt") as f:
    circuit = parse(f.read())
    dot = visualize_circuit(circuit)
    display(dot)

In [6]:
# type: ignore
def swap_wire_outputs(graph: nx.DiGraph, pairs):
    """Swap the output wires for given pairs of gates"""
    for w1, w2 in pairs:
        # Find the gates that output to these wires
        gate1 = next(g for g in graph.predecessors(
            next(n for n in graph.nodes if isinstance(n, Wire) and n.name == w1)))
        gate2 = next(g for g in graph.predecessors(
            next(n for n in graph.nodes if isinstance(n, Wire) and n.name == w2)))

        # Remove old edges
        for old_wire in list(graph.successors(gate1)):
            graph.remove_edge(gate1, old_wire)
        for old_wire in list(graph.successors(gate2)):
            graph.remove_edge(gate2, old_wire)

        # Add new edges
        graph.add_edge(gate1, next(n for n in graph.nodes if isinstance(n, Wire) and n.name == w2))
        graph.add_edge(gate2, next(n for n in graph.nodes if isinstance(n, Wire) and n.name == w1))

In [ ]:
def test_addition(graph: nx.DiGraph, x_val: int, y_val: int) -> tuple[int, int]:
    # Set x inputs
    for node in graph.nodes:
        if isinstance(node, Wire):
            if node.name.startswith('x'):
                bit_pos = int(node.name[1:])
                node.value = (x_val >> bit_pos) & 1
            elif node.name.startswith('y'):
                bit_pos = int(node.name[1:])
                node.value = (y_val >> bit_pos) & 1

    # Run simulation
    simulate_circuit(graph)

    # Get actual result
    actual = 0
    for n in graph.nodes:
        if isinstance(n, Wire) and n.name.startswith('z'):
            bit_pos = int(n.name[1:])
            if n.value:
                actual |= (1 << bit_pos)

    return actual, x_val + y_val


with open("../data/day24.txt") as f:
    circuit = parse(f.read())
    swap_pairs = [
        ('jmq', 'z06'),
        ('z38', 'qrh'),    # second pair we found
        ('rqf', 'cbd'),    # new pair from z13 area
        ('gmh', 'z13'),    # new pair from z45 area
    ]
    swap_wire_outputs(circuit, swap_pairs)

    # Validate
    # Test the same power-of-2 cases that revealed problems before
    test_cases = []
    for i in range(45):
        power = 2**i
        test_cases.append((power, 1))
        test_cases.append((power-1, 1))
        test_cases.append((power+1, 1))

    print("Testing powers of 2 after all swaps:")
    for x, y in test_cases:
        actual, expected = test_addition(circuit, x, y)
        if actual != expected:
            print(f"\nMismatch for {x:4d} + {y:4d}:")
            print(f"   Got:      {actual:4d} = {bin(actual)[2:]:>20}")
            print(f"   Expected: {expected:4d} = {bin(expected)[2:]:>20}")

    dot = visualize_circuit(circuit)
    display(dot)
